In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import itertools
from sklearn import linear_model
from numpy import ones,vstack
from numpy.linalg import lstsq
from datetime import datetime

In [2]:
df = pd.read_csv("data/Ames_Housing_Price_Data_raw.csv", index_col = 0)

In [3]:
df = df.drop_duplicates() # one house is duplicated, PID 909276070

In [4]:
# reset index bc data has rows with same indices, i.e. indices start over at 1 at certain points
df = df.reset_index(drop = True)

# Type dictionaries

In [5]:
typedict = {#'PID' : 'nominal',
            'SalePrice' : 'Continuous',
            #Matt
            'LotFrontage' : 'Continuous', 
            'LotArea' : 'Continuous',
            'maybe_LotShape' : 'Nominal',
            'LandSlope' : 'Nominal', 
            'LandContour' : 'Nominal', 
            'maybe_MSZoning' : 'Nominal', 
            'Street_paved' : 'Nominal', 
            'Alley' : 'Nominal',
            'Neighborhood' : 'Nominal', 
            #'drop_LotConfig' : 'nominal', 
            #'drop_Condition1' : 'nominal', 
            #'drop_Condition2' : 'nominal',
            'Foundation' : 'Nominal',
            'Utilities' : 'Nominal',
            'Heating' : 'Nominal',
            #'drop_HeatingQC_nom' : 'Ordinal',
            'CentralAir' : 'Nominal',
            #'drop_Electrical' : 'Nominal',
            'HeatingQC_ord' : 'Ordinal',
            'LotShape_com' : 'Nominal',
            'MSZoning_com' : 'Nominal',
            #'LF_Normal' : 'nominal',
            'LF_Near_NS_RR' : 'Nominal',
            'LF_Near_Positive_Feature' : 'Nominal',
            'LF_Adjacent_Arterial_St' : 'Nominal',
            'LF_Near_EW_RR' : 'Nominal',
            'LF_Adjacent_Feeder_St' : 'Nominal',
            'LF_Near_Postive_Feature' : 'Nominal',
            'Heating_com' : 'Nominal',
            'Electrical_com' : 'Nominal',
            'LotConfig_com' : 'Nominal', 
            'LotFrontage_log' : 'Continuous',
            'LotArea_log' : 'Continuous',
            #Oren 
            'MiscFeature': 'Nominal',
            'Fireplaces': 'Discrete',
            'FireplaceQu': 'Ordinal',
            'PoolQC': 'Ordinal',
            'PoolArea': 'Continuous',
            'PavedDrive': 'Nominal',
            'ExterQual': 'Ordinal',
            'OverallQual': 'Ordinal',
            'maybe_OverallCond': 'Ordinal',
            'MiscVal': 'Continuous',
            'YearBuilt': 'Discrete',
            'YearRemodAdd': 'Discrete',
            'KitchenQual': 'Ordinal',
            'Fence': 'Ordinal',
            'RoofStyle': 'Nominal',
            'RoofMatl': 'Nominal',
            #'maybe_Exterior1st': 'Nominal',
            #'drop_Exterior2nd': 'Nominal',
            'maybe_ExterCond': 'Ordinal',
            'maybe_MasVnrType': 'Nominal',
            'MasVnrArea': 'Continuous',
            #Mo
            #Basement
            'BsmtQual_ord': 'Ordinal',
            'BsmtCond_ord': 'Ordinal',
            'BsmtExposure_ord': 'Ordinal',
            #'BsmtQual_ord_lin': 'Ordinal',
            #'BsmtCond_ord_lin': 'Ordinal',
            #'BsmtExposure_ord_lin': 'Ordinal',
            'TotalBsmtSF': 'Continuous',
            'BSMT_GLQ':'Continuous', 
            'BSMT_Rec':'Continuous',
            'BsmtUnfSF': 'Continuous',
            'BSMT_ALQ':'Continuous',
            'BSMT_BLQ':'Continuous', 
            'BSMT_LwQ':'Continuous', 
            #'drop_BsmtQual': 'Nominal',
            #'drop_BsmtCond': 'Nominal',
            #'drop_BsmtExposure': 'Nominal',
            #'drop_BsmtFinType1': 'Nominal',
            #'drop_BsmtFinSF1': 'Continuous',
            #'drop_BsmtFinType2': 'Nominal',
            #'drop_BsmtFinSF2': 'Continuous',
            #Deck
            'WoodDeckSF':'Continuous', 
            'OpenPorchSF':'Continuous', 
            'ScreenPorch':'Continuous',
            'maybe_EnclosedPorch':'Continuous',
            'maybe_3SsnPorch':'Continuous',
            #Garage
            'GarageFinish':'Nominal', 
            'GarageYrBlt':'Continuous',
            'GarageCars':'Ordinal',
            'GarageArea':'Continuous',
            'GarageType_com':'Nominal',
            'GarageQual':'Nominal', 
            'GarageCond':'Nominal',
            #'drop_GarageType':'Nominal',

            # Hao-Wei
            "SaleType": "Nominal",
            "BldgType": "Nominal",
            "Functional_ord": "Ordinal", # Changed from "Functional"
            "1stFlrSF": "Continuous",
            "2ndFlrSF": "Continuous",
            "LowQualFinSF": "Continuous", # Rejectable p-value
            "GrLivArea": "Continuous",
            "BsmtFullBath": "Discrete",
            "BsmtHalfBath": "Discrete", # Rejectable p-value
            "FullBath": "Discrete",
            "HalfBath": "Discrete",
            "BedroomAbvGr": "Discrete",
            "KitchenAbvGr": "Discrete",
            "TotRmsAbvGrd": "Discrete",
            "MoSold": "Discrete", # Rejectable p-value
            "YrSold": "Discrete", # Rejectable p-value
            ####### Below are columns created by myself #######
            #"Functional_dis": "Discrete", # Functional in a (Salvage) 0-7 (Full) scale.
            "1stFlrSF_log": "Continuous",
            "2ndFlrSF_log": "Continuous",
            "GrLivArea_log": "Continuous",
            "number_floors": "Discrete",
            "attic": "Ordinal",
            "PUD": "Nominal",
            #### Whose?
            "SaleCondition": "Nominal",
            "SalePrice_log": "Continuous",
            #"drop_MS_Coded": "Nominal",
            "sold_datetime": "Discrete"
}

In [6]:
#Categorization of original variables
general=['PID','SalePrice']
lot_aspects=['LotFrontage','LotArea','LotShape','LandSlope','LandContour']
building_size=['MSSubClass','BldgType','HouseStyle','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd']
location=['MSZoning','Street','Alley','Neighborhood']
location_aspects=['LotConfig','Condition1','Condition2']
amenities=['MiscFeature','Fireplaces','FireplaceQu','PoolQC','PoolArea','PavedDrive']
garage=['GarageFinish','GarageType','GarageYrBlt','GarageCars','GarageArea','GarageQual','GarageCond']
decks=['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch']
basement=['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF']
utilities=['Foundation','Utilities','Heating','HeatingQC','CentralAir','Electrical']
quality_ratings=['ExterQual','OverallQual','OverallCond','MiscVal','YearBuilt','YearRemodAdd','KitchenQual','Fence','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','ExterCond','MasVnrType','MasVnrArea']
sales_aspect=['Functional','SaleCondition','SaleType','MoSold','YrSold']

# Matt

In [7]:
#add log Price column

df['SalePrice_log']=np.log10(df['SalePrice'])

In [8]:
# ordinalize heating quality ratings
HousingQC_dict={
       'Ex':5,
       'Gd':4,
       'TA':3,
       'Fa':2,
       'Po':1,
}

df.loc[df['HeatingQC'].isna(),'HeatingQC']='0'
df['HeatingQC_ord']=df['HeatingQC'].map(lambda x: HousingQC_dict[x])
df.rename(columns={'HeatingQC':'drop_HeatingQC'}, inplace=True)

In [9]:
# LotShape: combine IR2 (moderately irregular) and IR3 (irregular) into 'Irregular' due to small sample sizes
lot_shape_dict = {
    'Reg':'Regular',
    'IR1':'Slightly irregular',
    'IR2':'Irregular',
    'IR3': 'Irregular'
}
df['LotShape_com'] = df['LotShape'].map(lambda x: lot_shape_dict[x] if x in lot_shape_dict else x)
df.rename(columns={'LotShape':'drop_LotShape'}, inplace=True)

In [10]:
# LandSlope: combine Mod (moderate) and Sev (severe) into 'Moderate-severe' due to small sample sizes
land_slope_dict = {
    'Gtl':'Gentle',
    'Mod':'Moderate-severe',
    'Sev':'Moderate-severe'
}
df['LandSlope'] = df['LandSlope'].map(lambda x: land_slope_dict[x] if x in land_slope_dict else x)

In [11]:
# rename missing values in Alley column to 'No alley access'
alley_dict = {
    'Pave':'Paved',
    'Grvl':'Gravel',
    'No alley access' : 'No alley access'
}
df.loc[df['Alley'].isna(),'Alley'] = 'No alley access'
df['Alley'] = df['Alley'].map(lambda x: alley_dict[x] if x in alley_dict else x)

In [12]:
# simple renaming LandContour values for clarity
LandContour_dict = {
    'Lvl':'Level',
    'Bnk':'Banked (rise from street level to building)',
    'HLS' : 'Hillside (downward slope on both sides)',
    'Low' : 'Depression (upward slope on both sides)'
}

df['LandContour'] = df['LandContour'].map(lambda x: LandContour_dict[x] if x in LandContour_dict else x)

In [13]:
# Combine C(all) (commercial), I(all) (industrial), and A(agr) (agricultural) zoning types into 'Nonresidential' due to
# small sample sizes and the fact that we are focusing on residential sales
MSZoning_dict = {
    'RL':'Residential, low-density',
    'RM':'Residential, medium-density',
    'FV' : 'Residential, village',
    'RH' : 'Residential, high-density',
    'C (all)' : 'Nonresidential',
    'I (all)' : 'Nonresidential',
    'A (agr)' : 'Nonresidential'
}

df['MSZoning_com'] = df['MSZoning'].map(lambda x: MSZoning_dict[x] if x in MSZoning_dict else x)
df.rename(columns={'MSZoning':'drop_MSZoning'}, inplace=True)

In [14]:
# combine 'Near (within 200 ft)' and 'Adjacent to' into 'Near' for North-South RR, East-West RR, and positive features (parks, greenways, etc)
# renaming them LF_<factor> for Location Factor instead of condition to avoid confusion, as condition is also used to describe
# state of maintenance of various other features in the dataset
Condition_dict = {
    'Norm' : 'LF_Normal',
    'RRAn' : 'LF_Near_NS_RR',
    'PosN' : 'LF_Near_Positive_Feature',
    'Artery' : 'LF_Adjacent_Arterial_St',
    'RRAe' : 'LF_Near_EW_RR',
    'Feedr' : 'LF_Adjacent_Feeder_St',
    'PosA' : 'LF_Near_Postive_Feature',
    'RRNn' : 'LF_Near_NS_RR',
    'RRNe' : 'LF_Near_EW_RR'
}

df['Condition1'] = df['Condition1'].map(lambda x: Condition_dict[x] if x in Condition_dict else x)
df['Condition2'] = df['Condition2'].map(lambda x: Condition_dict[x] if x in Condition_dict else x)

In [15]:
def combine_condition_columns(df, factors):
    '''
    combines the "Condition1_com" and "Condition2_com" columns into a set of dummies for the values in those 2 columns
    '''
    for i in range(0, df.shape[0]):
        for factor in factors:
            if df.loc[i, 'Condition1'] == factor or df.loc[i, 'Condition2'] == factor:
                df.loc[i, f'{factor}'] = 1
            else:
                df.loc[i, f'{factor}'] = 0
    return df

In [16]:
loc_factors = Condition_dict.values()
df = combine_condition_columns(df, loc_factors)

In [17]:
# recommend drop condition columns
df.rename(columns={'Condition1':'drop_Condition1'}, inplace=True)
df.rename(columns={'Condition2':'drop_Condition2'}, inplace=True)
df.rename(columns={'LF_Normal':'drop_LF_Normal'}, inplace=True)

In [18]:
df.rename(columns={'Street':'Street_paved'}, inplace=True) # renaming 'Street' to 'Street_paved'

In [19]:
# Utilities: simple renaming for clarity
Utilities_dict = {
    'AllPub':'EGWS',
    'NoSewr':'EGW with septic tank'
}

df['Utilities'] = df['Utilities'].map(lambda x: Utilities_dict[x] if x in Utilities_dict else x)

In [20]:
# combine 'Gravity furnace', 'Other water/steam heating', 'Floor furnace', and 'Wall furnace' into 'Other' due to
# small sample size
Heating_dict = {
    'GasA':'Gas-powered forced-air heating',
    'GasW':'Gas-powered water/steam heating',
    'Grav' : 'Other',
    'OthW' : 'Other',
    'Floor' : 'Other',
    'Wall' : 'Other'
}

df['Heating_com'] = df['Heating'].map(lambda x: Heating_dict[x] if x in Heating_dict else x)
df.rename(columns={'Heating':'drop_Heating'}, inplace=True)

In [21]:
# renaming for clarity and combining FuseP and FuseF categories due to small sample size
# they are also the 2 most undesirable electrical setups as reported by the data dictionary
Electrical_dict = {
    'SBrkr': 'Standard circuit breakers, all Romex wiring',
    'FuseA': '>60 Amp fuse box, all Romex wiring',
    'FuseF' : '60 Amp fuse box, Romex or older wiring',
    'FuseP' : '60 Amp fuse box, Romex or older wiring'
}

df['Electrical_com'] = df['Electrical'].map(lambda x: Electrical_dict[x] if x in Electrical_dict else x)
df.rename(columns={'Electrical':'drop_Electrical'}, inplace=True)

In [22]:
# combined FR2 (2 sides frontage) and FR3 (3 sides frontage) into 2+ sides frontage due to small sample size
LotConfig_dict = {
    'Inside': 'Inside lot (1 side frontage)',
    'Corner': 'Corner lot',
    'CulDSac' : 'Cul-de-sac lot',
    'FR2' : '2+ sides frontage',
    'FR3' : '2+ sides frontage'
}

df['LotConfig_com'] = df['LotConfig'].map(lambda x: LotConfig_dict[x] if x in LotConfig_dict else x)
df.rename(columns={'LotConfig':'drop_LotConfig'}, inplace=True)

In [23]:
df['LotFrontage_log'] = np.log10(df['LotFrontage'])

In [24]:
df['LotArea_log'] = np.log10(df['LotArea'])

# Oren

In [25]:
Cond_dict={
       'Ex':5,
       'Gd':4,
       'TA':3,
       'Fa':2,
       'Po':1,
       'NA':0,
        '0':0
}

In [26]:
df.loc[df['ExterQual'].isna(),'ExterQual']='0'
df['ExterQual']=df['ExterQual'].map(lambda x: Cond_dict[x])

df.loc[df['ExterCond'].isna(),'ExterCond']='0'
df['ExterCond']=df['ExterCond'].map(lambda x: Cond_dict[x])

df.loc[df['KitchenQual'].isna(),'KitchenQual']='0'
df['KitchenQual']=df['KitchenQual'].map(lambda x: Cond_dict[x])

df.loc[df['FireplaceQu'].isna(),'FireplaceQu']='0'
df['FireplaceQu']=df['FireplaceQu'].map(lambda x: Cond_dict[x])

In [27]:
Paved_Drive_Dict={
       'Y':'Paved' ,
       'P':'Partial Pavement',
       'N':'Dirt Gravel'
}
df['PavedDrive']=df['PavedDrive'].map(lambda x: Paved_Drive_Dict[x] if x != 'NA' else x)

Fence_Dict={
       'GdPrv':'Good Privacy',
       'MnPrv':'Minimum Privacy',
       'GdWo':'Good Wood',
       'MnWw':'Minimum Wood/Wire',
       'NA':'No Fence'
}
df.loc[df['Fence'].isna(),'Fence'] = 'NA'
df['Fence']=df['Fence'].map(lambda x: Fence_Dict[x])

Misc_Feature_Dict={
       'Elev':'Elevator',
       'Gar2':'2nd Garage',
       'Othr':'Other',
       'Shed':'Shed',
       'TenC':'Tennis Court',
       'NA':'Nothing'
}
df.loc[df['MiscFeature'].isna(),'MiscFeature'] = 'NA'
df['MiscFeature']=df['MiscFeature'].map(lambda x: Misc_Feature_Dict[x])


Roof_Style_Dict={
       'Flat':'Flat',
       'Gable':'Gable',
       'Gambrel':'Gabrel Barn',
       'Hip':'Hip',
       'Mansard':'Mansard',
       'Shed':'Shed'
}
df['RoofStyle']=df['RoofStyle'].map(lambda x: Roof_Style_Dict[x] if x != 'NA' else x)  
    
    
Roof_Matl_Dict={
       'ClyTile':'Clay or Tile',
       'CompShg':'Standard (Composite) Shingle',
       'Membran':'Membrane',
       'Metal':'Metal',
       'Roll':'Roll',
       'Tar&Grv':'Gravel & Tar',
       'WdShake':'Wood Shakes',
       'WdShngl':'Wood Shingles'
}
df['RoofMatl']=df['RoofMatl'].map(lambda x: Roof_Matl_Dict[x] if x != 'NA' else x)    

# combined 'Asphalt Shingles', 'Stone', 'PreCast', and 'Cinderblock' as 'Other' due to small sample size
Exterior_Dict={
    'AsbShng':'ext_Asbestos_Shingles',
    'AsphShn':'ext_Other',
    'BrkComm':'ext_Common_Brick',
    'BrkFace':'ext_Face_Brick',
    'CBlock':'ext_Other',
    'CemntBd':'ext_Cement_Board',
    'CmentBd':'ext_Cement_Board',
    'HdBoard':'ext_Hard_Board',
    'ImStucc':'ext_Imitation_Stucco',
    'MetalSd':'ext_Metal_Siding',
    'Other':'ext_Other',
    'Plywood':'ext_Plywood',
    'PreCast':'ext_Other',
    'Stone':'ext_Other',
    'Stucco':'ext_Stucco',
    'VinylSd':'ext_Vinyl_Siding',
    'Wd Sdng':'ext_Wood_Siding',
    'WdShing':'ext_Wood_Shingles',
    'Wd Shng':'ext_Wood_Shingles',
    'Brk Cmn':'ext_Common_Brick'
}
df['Exterior1st']=df['Exterior1st'].map(lambda x: Exterior_Dict[x] if x != 'NA' else x)
df['Exterior2nd']=df['Exterior2nd'].map(lambda x: Exterior_Dict[x] if x != 'NA' else x)

Mas_Vnr_Type_Dict={
       'BrkCmn':'Brick Common',
       'BrkFace':'Brick Face',
       'CBlock':'Cinder Block',
       'None':'None',
       'Stone':'Stone'
}

df.loc[df['MasVnrType'].isna(),'MasVnrType'] = 'None'
df['MasVnrType']=df['MasVnrType'].map(lambda x: Mas_Vnr_Type_Dict[x] if x != 'NA' else x)

In [28]:
def combine_exterior_columns(df, factors):
    '''
    combines the "Condition1_com" and "Condition2_com" columns into a set of dummies for the values in those 2 columns
    '''
    for i in range(0, df.shape[0]):
        for factor in factors:
            if df.loc[i, 'Exterior1st'] == factor or df.loc[i, 'Exterior2nd'] == factor:
                df.loc[i, f'{factor}'] = 1
            else:
                df.loc[i, f'{factor}'] = 0
    return df

In [29]:
ext_values = ((pd.concat([df['Exterior1st'], df['Exterior2nd']], axis = 0)))

In [30]:
ext_factors = pd.unique(ext_values)

In [31]:
ext_factors

array(['ext_Wood_Siding', 'ext_Hard_Board', 'ext_Metal_Siding',
       'ext_Vinyl_Siding', 'ext_Wood_Shingles', 'ext_Plywood',
       'ext_Stucco', 'ext_Cement_Board', 'ext_Face_Brick',
       'ext_Asbestos_Shingles', 'ext_Common_Brick',
       'ext_Imitation_Stucco', 'ext_Other'], dtype=object)

In [32]:
df=combine_exterior_columns(df, ext_factors)

In [33]:
df.rename(columns={
    # Columns to drop/maybe
    'OverallCond': 'OverallCond',
    'Exterior2nd':'drop_Exterior2nd', 
    'Exterior1st': 'drop_Exterior1st',
}, inplace=True)

# Mo

In [34]:
def variable_selection(x):
    '''
    Iterate throuh all combunaions of variables and linearly regress to find optimal variables to utilize/ drop
    '''

    lm=linear_model.LinearRegression()

    for i in range(1,len(x.columns)):
        scores = {}

        for item in set(itertools.combinations(x.columns, i)):
            lm.fit(x[list(item)], df['SalePrice'])
            scores[item]=lm.score(x[list(item)], df['SalePrice'])

        print(scores[max(scores, key=lambda key: scores[key])])
        print(max(scores, key=lambda key: scores[key]))

In [35]:
#replace nominal with ordinal variables on standard scale with even steps

def linarization_func(var_name):
    '''
    Input: ordinal variable name as string
    Function creates new variable with naming *_lin that linarizes the ordinal scale 
    based on relationship to mean sales
    Variable needs to be part of a dataframe named df, which also includes oclumn 'SalePrice'
    '''

    #linear function between min and max of mean
    meanlist=df[['SalePrice',f'{var_name}']].groupby(f'{var_name}').agg('mean')

    points = [(0,min(meanlist['SalePrice'])),(1,max(meanlist['SalePrice']))]
    x_coords, y_coords = zip(*points)
    A = vstack([x_coords,ones(len(x_coords))]).T
    m, c = lstsq(A, y_coords, rcond=None)[0]

    #loop reassigning x: current mean, future mean(x_pos on lin function)
    dict={}

    dict[min(df[f'{var_name}'].unique())]=0
    dict[max(df[f'{var_name}'].unique())]=1

    for i in df[f'{var_name}'].unique():
        if not i in dict:
            dict[i]=(meanlist.loc[meanlist.index==i,'SalePrice'][i]-c)/m

    #new value mapping dictionary
    df[f'{var_name}_lin']=df[f'{var_name}'].map(lambda x: dict[x])


Basement

In [36]:
master_dict={
       'Ex':5,
       'Gd':4,
       'TA':3,
       'Fa':2,
       'Po':1,
       'NA':0,
        '0':0
}

exp_dict={
       'Gd':4,
       'Av':3,
       'Mn':2,
       'No':1,
       'NA':0,
        '0':0
}

In [37]:
#replace nominal with ordinal variables on standard scale with even steps
df['BsmtCond_ord']=df['BsmtCond']
df.rename(columns = {'BsmtCond': 'drop_BsmtCond'}, inplace=True)
df.loc[df['BsmtCond_ord'].isna(),'BsmtCond_ord']='0'
df['BsmtCond_ord']=df['BsmtCond_ord'].map(lambda x: master_dict[x])

df['BsmtQual_ord']=df['BsmtQual']
df.rename(columns = {'BsmtQual': 'drop_BsmtQual'}, inplace=True)
df.loc[df['BsmtQual_ord'].isna(),'BsmtQual_ord']='0'
df['BsmtQual_ord']=df['BsmtQual_ord'].map(lambda x: master_dict[x])

df['BsmtExposure_ord']=df['BsmtExposure']
df.rename(columns = {'BsmtExposure': 'drop_BsmtExposure'}, inplace=True)
df.loc[df['BsmtExposure_ord'].isna(),'BsmtExposure_ord']='0'
df['BsmtExposure_ord']=df['BsmtExposure_ord'].map(lambda x: exp_dict[x])

#drop 'unf' and 'NaN' dummies from BsmtFinType1 and BsmtFinType2 (unf covered through separate dumym already)
#need to merge dummies for BsmtFinType1 and BsmtFinType2
df['BSMT_GLQ']=0
df['BSMT_ALQ']=0
df['BSMT_BLQ']=0
df['BSMT_LwQ']=0
df['BSMT_Rec']=0

df.loc[df['BsmtFinType1'] == 'GLQ','BSMT_GLQ']=df.loc[df['BsmtFinType1'] == 'GLQ','BsmtFinSF1']
df.loc[df['BsmtFinType2'] == 'GLQ','BSMT_GLQ']=df.loc[df['BsmtFinType2'] == 'GLQ','BsmtFinSF2']

df.loc[df['BsmtFinType1'] == 'ALQ','BSMT_ALQ']=df.loc[df['BsmtFinType1'] == 'ALQ','BsmtFinSF1']
df.loc[df['BsmtFinType2'] == 'ALQ','BSMT_ALQ']=df.loc[df['BsmtFinType2'] == 'ALQ','BsmtFinSF2']

df.loc[df['BsmtFinType1'] == 'BLQ','BSMT_BLQ']=df.loc[df['BsmtFinType1'] == 'BLQ','BsmtFinSF1']
df.loc[df['BsmtFinType2'] == 'BLQ','BSMT_BLQ']=df.loc[df['BsmtFinType2'] == 'BLQ','BsmtFinSF2']

df.loc[df['BsmtFinType1'] == 'LwQ','BSMT_LwQ']=df.loc[df['BsmtFinType1'] == 'LwQ','BsmtFinSF1']
df.loc[df['BsmtFinType2'] == 'LwQ','BSMT_LwQ']=df.loc[df['BsmtFinType2'] == 'LwQ','BsmtFinSF2']

df.loc[df['BsmtFinType1'] == 'Rec','BSMT_Rec']=df.loc[df['BsmtFinType1'] == 'Rec','BsmtFinSF1']
df.loc[df['BsmtFinType2'] == 'Rec','BSMT_Rec']=df.loc[df['BsmtFinType2'] == 'Rec','BsmtFinSF2']

df.rename(columns = {'BsmtFinType1': 'drop_BsmtFinType1','BsmtFinSF1': 'drop_BsmtFinSF1','BsmtFinType2': 'drop_BsmtFinType2','BsmtFinSF2': 'drop_BsmtFinSF2'}, inplace=True)

df.loc[df['TotalBsmtSF'].isna(),'TotalBsmtSF']=0
df.loc[df['BsmtUnfSF'].isna(),'BsmtUnfSF']=0

#further columns I recommend we drop, based on them not having any effect by themselves on predicting sales prices
df.rename(columns = {'BsmtUnfSF': 'BsmtUnfSF','BSMT_ALQ': 'BSMT_ALQ','BSMT_BLQ': 'BSMT_BLQ','BSMT_LwQ': 'BSMT_LwQ','BsmtExposure': 'BsmtExposure'}, inplace=True)


Porches/ Decks

In [38]:
df.rename(columns = {'EnclosedPorch': 'EnclosedPorch','3SsnPorch': '3SsnPorch'}, inplace=True)


Garage

In [39]:
# consolidate Garage Types based on better predicitve power and low impact of other types
garagetype={
   'Detchd':'Detached', 
    'Attchd':'Attached', 
    'BuiltIn':'BuiltIn', 
    'Basment':'Basement',  
    '2Types':'2types', 
    'CarPort':'Detached',
    'No garage':'No garage'
}

In [40]:
df['GarageType_com']=df['GarageType']
df.rename(columns = {'GarageType': 'drop_GarageType'}, inplace=True)
df.loc[df['GarageType_com'].isna(),'GarageType_com']='No garage'
df['GarageType_com']=df['GarageType_com'].map(lambda x: garagetype[x])

# maybe drop GarageCond, GarageQual (basically no value, also almost all values are consolidated in one status)
df.rename(columns = {'GarageCond': 'GarageCond','GarageQual': 'GarageQual'}, inplace=True)

# keep year, area, Finish, cars as is, all have strong predictive power and do not seem to allow for easy consolidation


## Hao-Wei

There is an all-in-one pack function called `data_cleaning_part_2`.

In [41]:
MB_dict = {
    20: "1-Story",
    30: "1-Story",
    40: "1-Story",
    120: "1-Story",
    45: "1.5-Story",
    50: "1.5-Story",
    150: "1.5-Story",
    60: "2-Story",
    70: "2-Story",
    160: "2-Story",
    75: "2.5-Story",
    80: "SplitMulti",
    180: "SplitMulti",
    190: "2FamConv",
    85: "SptFoyer",
    90: "Duplex"
};
df["MS_coded"] = df["MSSubClass"].apply(lambda x: MB_dict[x])

In [42]:
def calc_floors(feat1, feat2):
    floors = []
    zipped = zip(feat1, feat2)
    for item in zipped: 
        if item[0] in ['1-Story', '1.5-Story', '2-Story', '2.5-Story']:
            ms_coded_dict = {
                '1-Story' : 1,
                '1.5-Story' : 1,
                '2-Story' : 2,
                '2.5-Story' : 2
            }
            floors.append(ms_coded_dict[item[0]])
        else:
            HouseStyle_dict = {
                '1Story' : 1,
                '1.5Fin' : 1,
                '1.5Unf' : 1,
                '2Story' : 2,
                '2.5Fin' : 2,
                '2.5Unf' : 2,
                'SLvl' : 2,
                'SFoyer' : 2
            }
            floors.append(HouseStyle_dict[item[1]])
    return floors

In [43]:
df['number_floors'] = calc_floors(df['MS_coded'], df['HouseStyle'])

In [44]:
def calc_attic(feat):
    attic = []
    for item in feat: 
        if not (item in ['1.5Fin', '1.5Unf', '2.5Fin', '2.5Unf']):
            attic.append('No attic')
        else:
            if 'Fin' in item:
                attic.append('Finished')
            if 'Unf' in item:
                attic.append('Unfinished')
    return attic

In [45]:
df['attic'] = calc_attic(df['HouseStyle'])

In [46]:
df['PUD'] = df['MSSubClass'].apply(lambda x: 1 if x in [120, 150, 160, 180] else 0)

In [47]:
df.loc[653,['BldgType']] = '2fmCon'

In [48]:
def data_cleaning_part_2(df):
    '''
    Input variable:
    housing: a compatible dataframe.
    
    Description:
    Assume that housing is the dataframe directly imported from person2.csv,
    the function fills up the NA values and add some columns Hao-Wei felt necessary.
    For an explanation of the columns, see the dictionary above and the original description file.
    '''
    df[['BsmtFullBath','BsmtHalfBath']] = df[['BsmtFullBath','BsmtHalfBath']].fillna(0);
    # df = df.drop("PID", axis = 1);
    #df = df.reset_index(drop= True);
    
    # Ordinal variable handling
    functionality_dict={
        "Typ": 7, # Typical Functionality
        "Min1": 6, # Minor Deductions 1
        "Min2": 5, # Minor Deductions 2
        "Mod": 4, # Moderate Deductions
        "Maj1": 3, # Major Deductions 1
        "Maj2": 2, # Major Deductions 2
        "Sev": 1, # Severely Damaged
        "Sal": 0, # Salvage only
    };
    df.loc[df["Functional"].isna(), "Functional"]='0';
    df["Functional_ord"]=df["Functional"].map(lambda x: functionality_dict[x]);
    df.rename(columns = {"Functional": "drop_Functional"},inplace=True); # 21
    
    # Adding columns with log scales

    temp = pd.DataFrame({"1stFlrSF_log": np.log10(df["1stFlrSF"]),
                        "2ndFlrSF_log": np.log10(df["2ndFlrSF"]+1), # +1 to avoid -inf
                        "GrLivArea_log": np.log10(df["GrLivArea"])});
    df = pd.concat([df, temp], axis = 1); # 24
    
    # Add weight columns for bathrooms
#     half_equiv = [0.3, 0.5];

#     temp_dict = {};
#     for eq in half_equiv:
#         temp_dict["BsmtEqBath_"+"{:.1f}".format(eq)] = df["BsmtFullBath"] + eq*df["BsmtHalfBath"];
#         temp_dict["EqBath_"+"{:.1f}".format(eq)] =  df["FullBath"] + eq*df["HalfBath"];

#     temp = pd.DataFrame(temp_dict);
#     df = pd.concat([df, temp], axis = 1); # 28

    # Extract nominal columns for better interpretation.
#     temp_dict = {};
#     temp_dict["1-Story"]    = df.apply(lambda x: x["MSSubClass"] in [20, 30, 40, 120], axis=1);
#     temp_dict["1.5-Story"]  = df.apply(lambda x: x["MSSubClass"] in [45, 50, 150], axis=1);
#     temp_dict["2-Story"]    = df.apply(lambda x: x["MSSubClass"] in [60, 70, 160], axis=1);
#     temp_dict["2.5-Story"]  = df.apply(lambda x: x["MSSubClass"] == 75, axis=1);
#     temp_dict["SplitMulti"] = df.apply(lambda x: x["MSSubClass"] in [80, 180], axis=1);
#     temp_dict["2FamConv"]   = df.apply(lambda x: x["MSSubClass"] == 190, axis=1);
#     temp_dict["SptFoyer"]   = df.apply(lambda x: x["MSSubClass"] == 85, axis=1);
#     temp_dict["Duplex"]     = df.apply(lambda x: x["MSSubClass"] == 90, axis=1);
#     temp_dict["Unfinished"] = df.apply(lambda x: x["MSSubClass"] == 190, axis=1) | df.apply(lambda x: x["HouseStyle"] in ["1.5Unf", "2.5Unf"], axis=1);
#     temp_dict["PUD"]        = df.apply(lambda x: x["MSSubClass"] in [120, 150, 160, 180], axis=1);
#     temp_dict["1Fam"]       = df.apply(lambda x: x["BldgType"] == "1Fam", axis=1);
#     temp_dict["TwnhsE"]     = df.apply(lambda x: x["BldgType"] == "TwnhsE", axis=1);
#     temp_dict["TwnhsI"]     = df.apply(lambda x: x["BldgType"] == "TwnhsI", axis=1);

    #temp = pd.DataFrame(temp_dict).astype(int);
    #df = pd.concat([df, temp], axis = 1); # 41, 42 if PID not dropped
    
    # Some of my personal selection
    cols_drop = ["MSSubClass", "HouseStyle", 'MS_coded']; # No more "BldgType"
    cols_maybe = ["LowQualFinSF", "BsmtHalfBath", "HalfBath"];
    
    col_dict = {};
    for dr in cols_drop:
        col_dict[dr] = "drop_" + dr;
    for dr in cols_maybe:
        col_dict[dr] = "" + dr;
    
    df.rename(columns=col_dict, inplace=True);
    
    return df;

df=data_cleaning_part_2(df)

In [49]:
def make_datetime(df):
    sold_datetime = []
    for i in range(len(df)):
        curr_sold = datetime(year = (df['YrSold'])[i], month = (df['MoSold'])[i], day = 1)
        sold_datetime.append(curr_sold)
    return pd.Series(sold_datetime)

In [50]:
df['sold_datetime'] = make_datetime(df)

In [51]:
#create linearized variables for these three prdinal variables
# linarization_func('BsmtCond_ord')
# linarization_func('BsmtQual_ord')
# linarization_func('BsmtExposure_ord')

# Final changes

In [52]:
cols = [c for c in df.columns if c[0:5] != 'drop_']
df=df[cols]

In [53]:
df.loc[df['PoolQC'].isnull(), 'PoolQC'] = 'NA'

In [54]:
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

In [55]:
# Property 903426160 has missing GarageYrBlt, GarageFinish, 
# GarageQual, and GarageCond, but present GarageCars, GarageType_com, and GarageArea
# Setting GarageCars and GarageArea to 0 for simplicity

In [56]:
df.loc[df['PID'] == 903426160, 'GarageType_com'] = 'No garage'

In [57]:
df.loc[df['PID'] == 903426160, 'GarageCars'] = 0

In [58]:
df.loc[df['PID'] == 903426160, 'GarageArea'] = 0

In [59]:
df['GarageQual'].fillna('0', inplace = True)
df['GarageCond'].fillna('0', inplace = True)
df['GarageFinish'].fillna('No garage', inplace = True)
df['GarageYrBlt'].fillna(-1, inplace = True)

In [60]:
# one car has missing GarageCars and GarageArea, all other garage variables
# signify that it has no garage
# set its garage variables as no garage
df.loc[df['PID'] == 910201180, 'GarageCars'] = 0
df.loc[df['PID'] == 910201180, 'GarageArea'] = 0
df.loc[df['PID'] == 910201180, 'GarageType_com'] = 'No garage'

In [61]:
# convert garage year built to garage age, -1 for no garage
df['Garage_age_years'] = 0 * len(df)
for i in range(len(df)):
    if df.loc[i, 'GarageYrBlt'] > 0:
        dif = df.loc[i, 'sold_datetime'] - datetime(year=int(df.loc[i, 'GarageYrBlt']), month = 1, day = 1)
        df.loc[i, 'Garage_age_years'] = (dif.days/365)
    else:
        df.loc[i, 'Garage_age_years'] = -1

In [62]:
# bin garage ages in 20-year intervals, first bin -2 to 0 avoids no garage being interpreted as numeric
df['Garage_age_bin'] = pd.cut(df['Garage_age_years'], [-2, 0, 20, 40, 60, 2000], labels = ['No garage', '0-20', '20-40', '40-60', '60+'])

In [63]:
# added house age at time of sale in years
df['house_age_years'] = 0 * len(df)

for i in range(len(df)):
    dif = df.loc[i, 'sold_datetime'] - datetime(year=df.loc[i, 'YearBuilt'], month = 1, day = 1)
    df.loc[i, 'house_age_years'] = (dif.days/365)

In [64]:
# added how long ago house received remodel or addition prior to sale, in years
# no remodel gets value of -1
df['Remod_age_years'] = 0 * len(df)
for i in range(len(df)):
    dif = df.loc[i, 'sold_datetime'] - datetime(year=df.loc[i, 'YearRemodAdd'], month = 1, day = 1)
    if df.loc[i, 'YearRemodAdd'] <= df.loc[i, 'YrSold']:
        if df.loc[i, 'YearRemodAdd'] != df.loc[i, 'YearBuilt']:
            df.loc[i, 'Remod_age_years'] = (dif.days/365)
        else:
            df.loc[i, 'Remod_age_years'] = -1
    else:
        df.loc[i, 'Remod_age_years'] = -1

In [65]:
# bin remodel ages in 15-year intervals, first bin -2 to 0 avoids no remodel being interpreted as numeric
df['Remod_age_bin'] = pd.cut(df['Remod_age_years'], [-2, 0, 15, 30, 45, 2000], labels = ['No remodel', '0-15', '15-30', '30-45', '45+'])

In [66]:
# ordinalize these 3 columns that were left as ratings strings
to_ordinals = ['GarageQual', 'GarageCond', 'PoolQC']
for col in to_ordinals:
    df[col] = df[col].map(lambda x: master_dict[x])

In [67]:
# one house is missing electrical, it was built in 2006, all houses built after 2000 have standard circuit
# breakers and romex wiring. Imputed.
df.loc[df['PID']==916386080, 'Electrical_com'] = 'Standard circuit breakers, all Romex wiring'

In [68]:
missing_frontage_indices = df.loc[df['LotFrontage'].isnull(), :].index

In [69]:
# estimate missing lot frontages by assuming square lot and calculating frontage from area and lot config
def calc_frontage(area, config):
    sides_dict = {
    'Corner lot' : 2,
    'Inside lot (1 side frontage)' : 1,
    '2+ sides frontage' : 2.5, # 2+ sides includes 2 and 3, averaging as 2.5
    'Cul-de-sac lot' : 1
    }
    perim = np.sqrt(area) * 4 # get perimeter from area
    oneside = perim/4 # assume square, get length of one side of lot
    sides = sides_dict[config] # get number of sides of lot with frontage from LotConfig
    frontage = oneside*sides # multiply length of sides * number of sides on road to get frontage
    return frontage

In [70]:
for idx in missing_frontage_indices:
    df.loc[idx, 'LotFrontage'] = calc_frontage(df['LotArea'][idx], df['LotConfig_com'][idx])

In [71]:
df['LotFrontage_log']=np.log(df['LotFrontage'])

In [72]:
# remove columns whose information has been moved somewhere else
df.drop(columns = ['YearRemodAdd', 'GarageYrBlt', 'YearBuilt'], inplace = True)

In [73]:
nulls = {}
for col in df.columns:
    if df[col].isnull().any():
        nulls[col] = sum(df[col].isnull())
nulls

{}

In [74]:
list(df.columns)

['PID',
 'GrLivArea',
 'SalePrice',
 'LotFrontage',
 'LotArea',
 'Street_paved',
 'Alley',
 'LandContour',
 'Utilities',
 'LandSlope',
 'Neighborhood',
 'BldgType',
 'OverallQual',
 'OverallCond',
 'RoofStyle',
 'RoofMatl',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'CentralAir',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Fireplaces',
 'FireplaceQu',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition',
 'SalePrice_log',
 'HeatingQC_ord',
 'LotShape_com',
 'MSZoning_com',
 'LF_Near_NS_RR',
 'LF_Near_Positive_Feature',
 'LF_Adjacent_Arterial_St',
 'LF_Near_EW_RR',
 'LF_Adj

# Save cleaned dataset

In [76]:
#df.to_csv('./data/ames_housing_price_data_v4.csv', index = False)